#### Activation:
relu, sigmoid, tanh

#### weight initializer:
he, xavier, normal

#### optimizer:
Adam, Momemtum, Gradient Descent

In [1]:
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#读数据：
import os
import pickle as cPickle
import numpy as np

CIFAR_DIR = "../../../other_datasets/cifar-10-batches-py"

In [3]:
#将数据读取,返回data 和 labels
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding = 'latin1')
        return data['data'], data['labels']

In [4]:
x = tf.placeholder(tf.float32, [None, 3072])
x_image = tf.reshape(x, [-1, 3, 32, 32])
x_image = tf.transpose(x_image, perm = [0, 2, 3, 1]) #变成[-1, 32, 32, 3]
#x_image： 32 * 32
y = tf.placeholder(tf.int64, [None])

#### 将可变activation功能封装：

In [ ]:
def convnet(inputs, activation, kernel_initializer):
    conv1_1 = tf.layers.conv2d(inputs,
                         32, #output channel number
                         (3,3), #kenel size
                          padding = 'same', #使得输出和我的输入一样大，如果不想padding就padding = 'valid'
                          activation= activation,
                          kernel_initializer = kernel_initializer, #默认是tf.glorot_uniform_initializer
                          name = 'conv1_1' # 输出的时候，看名字我们知道是哪个层，好调试
                         )

    conv1_2 = tf.layers.conv2d(conv1_1,
                             32, #output channel number
                             (3,3), #kenel size
                              padding = 'same', #使得输出和我的输入一样大，如果不想padding就padding = 'valid'
                              activation= activation,
                              kernel_initializer = kernel_initializer, #默认是tf.glorot_uniform_initializer
                              name = 'conv1_2' # 输出的时候，看名字我们知道是哪个层，好调试
                             )
    pooling1 = tf.layers.max_pooling2d(conv1_2, # 输入是conv1
                                      (2,2), # kernel size
                                      (2,2), #步长：至少大于kernel size才能达到图像信息压缩
                                       name = 'pool1' # 输出的时候，看名字我们知道是哪个层，好调试
                                      )

    conv2_1 = tf.layers.conv2d(pooling1,
                             32, #output channel number
                             (3,3), #kenel size
                              padding = 'same', #使得输出和我的输入一样大，如果不想padding就padding = 'valid'
                              activation= activation,
                              kernel_initializer = kernel_initializer,
                              name = 'conv2_1' # 输出的时候，看名字我们知道是哪个层，好调试
                             )

    conv2_2 = tf.layers.conv2d(pooling1,
                             32, #output channel number
                             (3,3), #kenel size
                              padding = 'same', #使得输出和我的输入一样大，如果不想padding就padding = 'valid'
                              activation= activation,
                              kernel_initializer = kernel_initializer,
                              name = 'conv2_2' # 输出的时候，看名字我们知道是哪个层，好调试
                             )

    pooling2 = tf.layers.max_pooling2d(conv2_2, # 输入是conv1
                                      (2,2), # kernel size
                                      (2,2), #步长：至少大于kernel size才能达到图像信息压缩
                                       name = 'pool2' # 输出的时候，看名字我们知道是哪个层，好调试
                                      )

    conv3_1 = tf.layers.conv2d(pooling2,
                             32, #output channel number
                             (3,3), #kenel size
                              padding = 'same', #使得输出和我的输入一样大，如果不想padding就padding = 'valid'
                              activation= activation,
                              kernel_initializer = kernel_initializer,
                              name = 'conv3_1' # 输出的时候，看名字我们知道是哪个层，好调试
                             )

    conv3_2 = tf.layers.conv2d(pooling2,
                             32, #output channel number
                             (3,3), #kenel size
                              padding = 'same', #使得输出和我的输入一样大，如果不想padding就padding = 'valid'
                              activation= activation,
                              kernel_initializer = kernel_initializer,
                              name = 'conv3_2' # 输出的时候，看名字我们知道是哪个层，好调试
                             )

    pooling3 = tf.layers.max_pooling2d(conv3_2, # 输入是conv1
                                      (2,2), # kernel size
                                      (2,2), #步长：至少大于kernel size才能达到图像信息压缩
                                       name = 'pool3' # 输出的时候，看名字我们知道是哪个层，好调试
                                      )
    flatten = tf.layers.flatten(pooling3)
    return flatten


In [7]:
#flatten = convnet(x_image, tf.nn.relu) #train 10k: 73.35%
#flatten = convnet(x_image, tf.nn.sigmoid) #train 10k: 53.59%

#truncated_normal_initializer:
#均值是0，标准差是0.02, 但是这个会保证我们的值不会大于某个数，小于某个数。保证数字在合理范围
flatten = convnet(x_image, tf.nn.relu, tf.truncated_normal_initializer(stddev = 0.02)) 

#其他方法：基于tf.nn.relu
#1. 默认初始化：tf.glorot_uniform_initializer: 76.53% after 100k train
#2. tf.truncated_normal_initializer: 78.04% after 100k train
#3. tf.keras.initializers.he_normal: 71.52% after 100k train (在这里表现不好，可能是因为我们现在的网络较浅，网络深的时候何凯明的方法会更好)

Instructions for updating:
Use keras.layers.Dense instead.


In [ ]:
y_ = tf.layers.dense(flatten, 10) #输出10分类
#最后test准确率是：[Test] Step: 10000, acc: 0.72500

In [8]:
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)

In [9]:
#bool:
#predict = p_y_1 > 0.5

#indexes(int):
#第一个维度是同一类别下每个样本，第二维度是同一个样本下的每个类别（一个类别有一个概率值）
predict_indexes = tf.argmax(y_, 1) #1代表第二个维度，求一个样本，哪个类别的概率值最大

#int64:
#predict_float = tf.cast(predict, tf.int64)

#bool:
correct_prediction = tf.equal(predict_indexes, y) #两个int64的array比较

#float64:
correct_prediction_float = tf.cast(correct_prediction, tf.float32)

#float64, 因为correct_prediction_float是[1.0, 0.0, 1.0, ...] 而reduce_mean就是看有多少1，然后除以len
accuracy = tf.reduce_mean(correct_prediction_float)

### Optimizer:
1. tf.train.AdamOptimizer(1e-3).minimize(loss) #

2. GradientDescentOptimizer(1e-4).minimize(loss):这个初始化要小一点，否则loss非但不会缓慢下降，还会上升: 12.35% train 100k

3. MomentumOptimizer(learning_rate = 1e-4, momentum = 0.9).minimize(loss) : 37.75% train 100k

4. 为何修改为GD和M后optimizer不如Adam表现的好？随机表现都有1/10=10%的准确率：

4.1. initializer incorrect（NN是一个系统工程，我们换了Optimizer，相应的intializer也要换）

4.2. 不充分的训练（Adam训练比较快，所以100k就够了。可能GD,M需要更多次的训练，才能超过Adam的效果）

In [10]:
with tf.name_scope('train_op'):
    #train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
    #train_op = tf.train.GradientDescent(1e-4).minimize(loss)
    train_op = tf.train.MomentumOptimizer(learning_rate = 1e-4, momentum = 0.9).minimize(loss)

#### 使用tensorboard
1. 第一步：指定面板图上显示的变量，为了防止显示太多过载

2. 面板图的变量是tensorflow某个网络节点的值，所以在网络构建tf.name_scope()之后，来进行第一步

3. tensorboard的api在tf.summary中，其中标量使用scaler

In [11]:
#最重要的想输出的给tensorboard的是loss 和 accuracy
#因为是标量，所以用tf.summary.scaler()
#tf.summary.scaler()存储方法: 'loss' : <10, 1.1>, <20, 1.08>
#'loss'是key，然后<10, 1.1>意思是在第10次迭代的时候loss = 1.1

loss_summary = tf.summary.scalar('loss', loss)
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [12]:
#给输入图像建立summary，因为图像是3通道的32*32图像，所以使用tf.summary.image()
#不过注意，在CifarData中，我们将图像像素点归一化了，从[0,255]变成[0,1], 参见code：self._data = self._data / 127.5 - 1 
#所以我们不能直接使用x_image, 因为它是[0,1], 而是要逆归一化：
source_image = (x_image + 1) * 127.5
inputs_summary = tf.summary.image('inputs_image', source_image)

In [13]:
# 多给一些变量做summary，让它输出到tensorboard
# 功能性函数，打印变量的各种统计值：
def variable_summary(var, name):
    """Constructs summary for statistics of a variable"""
    #var: 计算summary的变量
    #name: 指定命名空间，防止冲突
    with tf.name_scope(name):
        mean = tf.reduce_mean(var) #这个var可以是一个大矩阵，但这个reduce_mean可以计算出全部的均值，返回一个值
        with tf.name_scope('std_dev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square( var - mean )))
            
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('min', tf.reduce_min(var)) #矩阵的全局的min
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.histogram('histogram', var) #变量的直方图
        

#将每个卷积层的各种统计值打印到tensorboard
with tf.name_scope('summary'):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')

In [14]:
# 可能会生成许多变量，导致sess.run()中的变量很长
# 采用merge_all(), merge() 解决

#用于train
merged_summary = tf.summary.merge_all() #合并所有之前tf.summary.xx(), 包括我们variable_summary()中的各种统计值

#用于test
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary]) #个性化合并

In [15]:
# 我们需要指定一个文件夹
LOG_DIR = '.' #这是当前文件夹
run_label = 'run_vgg_tensorboard' #这是子文件夹
run_dir = os.path.join(LOG_DIR, run_label) #子文件夹的总路径
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
    
# 分别设置两个文件夹
train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')

if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

#### 使用tensorboard
1. 第二步：训练过程中将这些变量计算出来。输出到dir的文件中

### cifar10 数据处理
因为mini-batch，所以需要在cifar10上循环遍历数据，每次在sess.run()的feed_dict塞入不同数据

In [16]:
#filenames: 对 训练数据集 和 测试数据集 是分开的
#need_shuffle: 训练数据集需要shuffle，测试数据集不需要shuffle
class CifarData:
    def __init__(self, filenames, need_shuffle):
        #读入数据：
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            #data的对应位置的图像的label，也是label对应位置的label，所以把它们两个zip起来
            all_data.append(data)
            all_labels.append(labels)
            #不再是只调取0，1类 
#             for item, label in zip(data, labels):
#                 if label in [0,1]:
#                     all_data.append(item)
#                     all_labels.append(label)
        #data是numpy的矩阵，
        #item是numpy的向量
        #all_data里面包含了许多numpy的向量，现在我们将这些向量纵向合并到一起，成为一个矩阵：用np.vstack()
        #all_labels里面包含了许多label的向量，现在我们将这些向量横向合并到一起，成为一个矩阵（但是这个矩阵只有一行：n * 1）：用np.hstack()
        self._data = np.vstack(all_data) 
        self._data = self._data / 127.5 - 1 #最后做的改进：将0-255的值二分成0，1
        self._labels = np.hstack(all_labels)
        #print(self._data.shape)
        #print(self._labels.shape)
        self._num_examples = self._data.shape[0] #多少个样本
        self._need_shuffle = need_shuffle
        self._indicator = 0 #遍历到数据集的哪个位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self): #在训练集上shuffle
        # np.random.permutation(6) -> [3,2,5,1,4,0]
        p = np.random.permutation(self._num_examples) #得到一个混的全排
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size): #每次返回batch_size个数个样本
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data() #就算遍历完，可是如果可以shuffle，我们的数据可以复用
                self._indicator = 0
                end_indicator = batch_size + 0
            else:
                raise Exception("have no more examples") #已经遍历完了并且不能shuffle
        if end_indicator > self._num_examples:
            #此时说明我们输入的batch_size比我们的所有样本都要大
            raise Exception("batch size is larger than example size")
        batch_data = self._data[self._indicator : end_indicator]
        batch_labels = self._labels[self._indicator : end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

### 测试我们的CifarData
1. 首先cifar10拥有50k个样本（10个类别），我们现在是二分类问题，只选取label为0，1的
2. 所以应该有 50k * (1/5) = 10k个样本 (10000, 3072)

In [17]:
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch') ]

train_data = CifarData(train_filenames, True)

In [18]:
test_data = CifarData(test_filenames, False)

In [19]:
batch_data, batch_labels = train_data.next_batch(3)

### 执行流程图

In [20]:
init = tf.global_variables_initializer() #执行初始化
#开启Session，相当于打开绘画
#sess.run()执行计算图
#1. 计算目标:
#表示要训练：[loss, accuracy, train_op]
#表示要测试：[loss, accuracy]

#2. feed_dict:塞入数据
# x: x要塞的数据
# 因为mini-batch，所以需要在cifar10上循环遍历数据，每次塞入不同数据

batch_size = 20 #每次取20张图来 训练 或者 测试
train_steps = 10000
test_steps = 100

output_summary_every_steps = 100 #因为merged_summary 和 merged_summary_test要计算的东西很多，所以不是每一步都计算

with tf.Session() as sess:
    sess.run(init)
    
    #FileWriter: 打开一个文件，在这个文件中写数据
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph) #sess.graph就是我们上面写的conv1， conv2 一大堆的东西的图
    test_writer = tf.summary.FileWriter(test_log_dir)

    #每次在测试的时候，我们希望我们都是在同一个test数据集下测试，而不是像训练数据集，每次都用不同的train数据集训练
    fixed_test_batch_data, fixed_test_batch_labels \
        = test_data.next_batch(batch_size)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss, accuracy, train_op]
        should_output_summary = ((i+1) % output_summary_every_steps == 0)
        if should_output_summary :
            eval_ops.append(merged_summary)
        
        #因为train_op返回的东西没有意义，所以用 _ 下划线接住返回值
        eval_ops_results = sess.run(
            eval_ops,  #这个是变长的
            feed_dict = {
                x: batch_data, 
                y: batch_labels})  #注意格式！
        loss_val, acc_val = eval_ops_results[0:2] #不管eval_ops_results长度是2还是3，这个都是合法的
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i + 1) #第二个参数 i+1，是告诉writer是哪一个步数下的train_summary_str
            #与此同时：计算merged_summary_test，同时采用固定的测试数据集
            test_summary_list = sess.run(
                                    [merged_summary_test],
                                     feed_dict = {
                                         x: fixed_test_batch_data,
                                         y: fixed_test_batch_labels
                                     })
            test_summary_str = test_summary_list[-1] #注意sess.run()返回的是list，我们需要的是第一个元素
            test_writer.add_summary(test_summary_str, i+1)
        #===============================
        #======训练=====================
        if (i+1) % 1000 == 0: #使用(i+1)就可以输出500和1000
            print("[Train] Step: %d, loss: %4.5f, acc: %4.5f"\
                 % (i+1, loss_val, acc_val))
        #===============================
        #======测试=====================
        if (i+1) % 5000 == 0: #使用(i+1)就可以输出500和1000
            test_data = CifarData(test_filenames, False)
            all_test_acc_vals = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    })
                all_test_acc_vals.append(test_acc_val)
            test_acc = np.mean(all_test_acc_vals)
            print("[Test] Step: %d, acc: %4.5f" % (i+1, test_acc))
            
            


[Train] Step: 1000, loss: 1.25953, acc: 0.45000
[Train] Step: 2000, loss: 1.50925, acc: 0.60000
[Train] Step: 3000, loss: 0.90767, acc: 0.75000
[Train] Step: 4000, loss: 0.94315, acc: 0.65000
[Train] Step: 5000, loss: 0.99022, acc: 0.70000
[Test] Step: 5000, acc: 0.68250
[Train] Step: 6000, loss: 1.37639, acc: 0.55000
[Train] Step: 7000, loss: 0.97922, acc: 0.55000
[Train] Step: 8000, loss: 0.56823, acc: 0.75000
[Train] Step: 9000, loss: 1.06372, acc: 0.60000
[Train] Step: 10000, loss: 0.68338, acc: 0.80000
[Test] Step: 10000, acc: 0.72950
